In [ ]:
import os
import uuid
import json
import logging
from dotenv import load_dotenv

from dapr_agents import OpenAIChatClient
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from dapr_agents.tool import AgentTool

logging.basicConfig(level=logging.INFO)
load_dotenv()

In [ ]:
llm = OpenAIChatClient(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

def to_obj(x):
    return json.loads(x) if isinstance(x, str) else x

In [ ]:
# ---- MCP Jupyter server env (REMOTE) ----
NOTEBOOK_NAME = f"mcp_{uuid.uuid4().hex[:8]}.ipynb"

ENV = {
    **os.environ,
    "MCP_JUPYTER_SESSION_MODE": "server",
    "MCP_JUPYTER_BASE_URL": os.getenv("JUPYTER_BASE_URL"),
    "MCP_JUPYTER_TOKEN": os.getenv("JUPYTER_TOKEN"),
    "MCP_JUPYTER_KERNEL_NAME": os.getenv("JUPYTER_KERNEL_NAME", "python3"),
    "MCP_JUPYTER_NOTEBOOK_PATH": os.getenv("JUPYTER_NOTEBOOK_PATH", NOTEBOOK_NAME),
}

# Adjust path to your MCP server entrypoint (where your server.py lives)
server_params = StdioServerParameters(
    command="uvx",
    args=[ "mcp-jupyter-notebook", "--transport", "stdio"],
    env=ENV,
)

In [ ]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as s:
        await s.initialize()

        # Discover tools
        tools = await AgentTool.from_mcp_session(s)

        from dapr_agents import Agent

        notebook_agent = Agent(
            name="Jupyter Notebook Orchestrator",
            role="A notebook co-pilot that drives analysis inside a persistent Jupyter session.",
            goal=(
                "Use the active Jupyter notebook to complete tasks end-to-end: "
                "add clear markdown, execute Python code, and, when necessary, install Python packages. "
                "Reuse the same live session (kernel + document) so work builds on previous cells."
            ),
            instructions=[
                # Session & ordering
                "You are driving a live Jupyter notebook (persistent kernel + document).",
                "Cell order matters. Issue exactly ONE tool call at a time and wait for its result before the next.",
                "Append new cells; do not rewrite or reorder existing cells unless explicitly asked.",

                # Tools to use
                "Use `notebook.markdown.add(content)` for headings and short explanations.",
                "Use `notebook.code.run(content)` to append and execute Python code cells.",

                # Packages
                "If an import fails, call `notebook.packages.add([pip_names])` once, then retry the import.",
                "Package names are pip distributions (e.g., 'matplotlib', 'seaborn', 'plotly', 'requests').",

                # Notebook hygiene
                "Before a task, add a brief markdown heading describing what you'll do.",
                "Keep code cells small and focused—one idea per cell.",
                "When previewing data, show concise outputs (e.g., df.head(), shapes, key metrics).",
                "For plots, include a title, axis labels, and legends where helpful.",

                # Robustness
                "If an error occurs, add a short markdown note on the fix, then run a corrected code cell.",
                "Reuse variables and data created in earlier cells; reference them directly in later cells.",

                # Closure
                "End a logical section with a short markdown summary of what was produced."
            ],
            llm=llm,
            tools=tools,
            max_iterations=20,
        )

        QUESTION = """create a matplotlib bar and line chart examples in the notebook"""
        await notebook_agent.run(QUESTION)